# ***Stable Diffusion 1.5***                                                          
**" Diffusion is still alive in Colab " -LBG**

# **Dependancies takes a while to setup dont panic just relax😊**

In [ ]:
# Skip if its your second time using Sd from Alive_Sd.ipynb

!pip install transformers diffusers accelerate
!pip install xformers # for memory management

# **For Text to Image gernation 🗨️_📷**






In [ ]:
'''
You can change "model_id" for your custom model just search on google your
model name  with last word "hugging face". copy model "model_id" from  huggingface site given example
code.

'''

from  diffusers import AutoPipelineForText2Image  # importing text to img pipeline
from google.colab import files   #some other imports (usefull)
import torch
import random
import matplotlib.pyplot as plt
import matplotlib.image as render

model_id = "emilianJR/epiCRealism"     # from huggingface sample code (Model)

pipe = AutoPipelineForText2Image.from_pretrained(

        # for gpu ⬇️
        model_id, torch_dtype=torch.float16, use_safetensors=True, safety_checker = None, requires_safety_checker = False

        # for cpu ⬇️
        # model_id, use_safetensors=True, safety_checker = None, requires_safety_checker = False
)


# efficent only for gpu ⬇️
pipe.enable_xformers_memory_efficient_attention()

pipe = pipe.to("cuda") # set "cpu" for cpu and "cuda" for gpu

# **Enter Positive and Negative Prompts Here ⬇️**

In [ ]:
# NOTE: NSFW filter is off by defult be alert !!!

prompts = "dog sitting on (bed)++"
negative = "blurry, bad_quality, worst_quality, bad_anotomy, extra_fingures, low_resolution"

# **Image Gernation With Custom Properties ☸️🛞**

In [ ]:
image = pipe(

             prompt = prompts,
             negative_prompt = negative,

             # you can change the below properties as per your wish ⬇️⬇️

             guidance_scale=7,         # how much image will close to prompt.
             num_inference_steps=50,   # higher value more quality but slow image gernations.
             height=720,
             weight=512,               # defult 512x512, for sdxl use 1024x1024

            ).images[0]

# code for showing and downloading gernated image.
imgName = f'{prompts}{random.randint(0,9999)}.png'
image = image.save(imgName)
output = render.imread(imgName)
plt.imshow(output)
plt.axis('off')         # to hide the graph axis (x & y)
plt.show()              # render img

usr = str(input("Download Image TYPE YES: "))
usr.lower()
if usr == "yes":
  files.download(imgName)
else:
  print(f"image is saved in /content/{imgName}")

# **Custom Api**

In [ ]:
!pip install flask
!pip install pyngrok

from flask import Flask, request, jsonify, send_from_directory
import torch
from diffusers import AutoPipelineForText2Image
import random
import os
from pyngrok import ngrok

# Authenticate ngrok with your authtoken
ngrok_auth_token = "Your Ngrok Token HERE !"  # Replace with your actual ngrok authtoken
ngrok.set_auth_token(ngrok_auth_token)

# Initialize the image generation pipeline
pipe = AutoPipelineForText2Image.from_pretrained(
    "emilianJR/epiCRealism",
    torch_dtype=torch.float16,
    use_safetensors=True,
    safety_checker=None,
    requires_safety_checker=False
)
pipe.enable_xformers_memory_efficient_attention()
pipe = pipe.to("cuda")  # Use GPU

# Flask app setup
app = Flask(__name__)

# Folder to save generated images
output_folder = '/content/generated_images'
os.makedirs(output_folder, exist_ok=True)

# Create a single ngrok tunnel when the app starts (do not call it inside the route)
ngrok.kill()  # Kill any previous tunnels
public_url = ngrok.connect(5000)  # This starts only one tunnel
print(f"Public URL for Flask app: {public_url}")

# This route should accept POST requests
@app.route('/generate', methods=['POST'])
def generate_image():
    # Receive the prompt and negative prompt from WebUI
    data = request.get_json()
    prompt = data.get("prompt", "")
    negative_prompt = data.get("negative_prompt", "")

    # Generate the image
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        guidance_scale=7,
        num_inference_steps=50,
        height=720,
        width=512
    ).images[0]

    # Save the generated image
    img_name = f"{prompt.replace(' ', '_')}_{random.randint(0, 9999)}.png"
    img_path = os.path.join(output_folder, img_name)
    image.save(img_path)

    # Return only the image file name (not the full ngrok URL)
    return jsonify({"image_name": img_name})

# Serve generated images dynamically
@app.route('/generated_images/<filename>')
def serve_image(filename):
    return send_from_directory(output_folder, filename)

# Start the Flask app
if __name__ == '__main__':
    app.run(port=5000, host='0.0.0.0')


# **More Information Here -:**


--> For further learning Stable Diffusion via code here you can Start from:

https://huggingface.co/docs/diffusers/en/using-diffusers/conditional_image_generation


**Future Updates:**

--> Stable Diffusion XL


---


--> Img to Img


---


--> Inpainting


---


--> MaskEditor for Inpainting


---


--> Loars


---


--> Face Warps


---


--> Control Net


---




***VERSION 1.0***